In [3]:
import fml_manager
import json
import time
import requests
import os

manager = fml_manager.FMLManager()

In [6]:
response = manager.load_data("./data/breast_b.csv", "fate_flow_test_breast", "breast_b", work_mode=1, head=1, partition=10)
output = json.loads(response.content)
print(output)
guest_job_id = output["jobId"]
guest_query_condition = {
    "job_id":guest_job_id
}

manager.query_job_status(guest_query_condition)

{'data': {'board_url': 'http://fateboard:8080/index.html#/dashboard?job_id=2020063006505834637941&role=local&party_id=0', 'job_dsl_path': '/data/projects/fate/python/jobs/2020063006505834637941/job_dsl.json', 'job_runtime_conf_path': '/data/projects/fate/python/jobs/2020063006505834637941/job_runtime_conf.json', 'logs_directory': '/data/projects/fate/python/logs/2020063006505834637941', 'namespace': 'fate_flow_test_breast', 'table_name': 'breast_b'}, 'jobId': '2020063006505834637941', 'retcode': 0, 'retmsg': 'success'}
Status: waiting
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: success


'success'

In [9]:
pipline = '''
{
    "components" : {
        "dataio_0": {
            "module": "DataIO",
            "input": {
                "data": {
                    "data": [
                        "args.train_data"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["dataio"]
            },
                        "need_deploy": true
         },
        "hetero_feature_binning_0": {
            "module": "HeteroFeatureBinning",
            "input": {
                "data": {
                    "data": [
                        "dataio_0.train"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["hetero_feature_binning"]
            }
        },
        "hetero_feature_selection_0": {
            "module": "HeteroFeatureSelection",
            "input": {
                "data": {"data": [
                        "hetero_feature_binning_0.train"
                    ]
                },
                "isometric_model": [
                    "hetero_feature_binning_0.hetero_feature_binning"
                ]
            },
            "output": {
                "data": ["train"],
                "model": ["selected"]
            }
        },
        "hetero_lr_0": {
            "module": "HeteroLR",
            "input": {
                "data": {
                    "train_data": ["hetero_feature_selection_0.train"]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["hetero_lr"]
            }
        },
        "evaluation_0": {
            "module": "Evaluation",
            "input": {
                "data": {
                    "data": ["hetero_lr_0.train"]
                }
            },
            "output": {
                "data": ["evaluate"]
            }
        }
    }
}
'''

config = '''
{
    "initiator": {
        "role": "guest",
        "party_id": 10000
    },
    "job_parameters": {
        "work_mode": 1
    },
    "role": {
        "guest": [10000],
        "host": [9999],
        "arbiter": [9999]
    },
    "role_parameters": {
        "guest": {
            "args": {
                "data": {
                    "train_data": [{"name": "breast_b", "namespace": "fate_flow_test_breast"}]
                }
            },
            "dataio_0":{
                "with_label": [true],
                "label_name": ["y"],
                "label_type": ["int"],
                "output_format": ["dense"]
            }
        },
        "host": {
            "args": {
                "data": {
                    "train_data": [{"name": "breast_a", "namespace": "fate_flow_test_breast"}]
                }
            },
             "dataio_0":{
                "with_label": [false],
                "output_format": ["dense"]
            }
        }
    },
    "algorithm_parameters": {
        "hetero_lr_0": {
            "penalty": "L2",
            "optimizer": "rmsprop",
            "eps": 1e-5,
            "alpha": 0.01,
            "max_iter": 3,
            "converge_func": "diff",
            "batch_size": 320,
            "learning_rate": 0.15,
            "init_param": {
                                "init_method": "random_uniform"
            }
        }
    }
}
'''

In [10]:
response = manager.submit_job(json.loads(pipline),json.loads(config))
manager.prettify(response, True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    "job_id":jobId
}

model_id, model_version = "", ""
final_status = manager.query_job_status(query_condition)

if final_status == "failed":
    print("Failed")
    manager.prettify(job_detail, True)
    response = manager.fetch_job_log(jobId)
    raise Exception("Failed to run the job")
if final_status == "success":
    print("Success")
    manager.prettify(response, True)
    output = json.loads(response.content)
    model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020063006531172788744&role=guest&party_id=9999",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020063006531172788744/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020063006531172788744/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020063006531172788744",
        "model_info": {
            "model_id": "arbiter-9999#guest-9999#host-9999#model",
            "model_version": "2020063006531172788744"
        }
    },
    "jobId": "2020063006531172788744",
    "retcode": 0,
    "retmsg": "success"
}
Status: waiting
Status: waiting
Status: waiting
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: run

In [21]:
pipline = '''
{
    "components" : {
        "dataio_0": {
            "module": "DataIO",
            "input": {
                "data": {
                    "data": [
                        "args.train_data"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["dataio"]
            },
                        "need_deploy": true
         },
        "hetero_feature_binning_0": {
            "module": "HeteroFeatureBinning",
            "input": {
                "data": {
                    "data": [
                        "dataio_0.train"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["hetero_feature_binning"]
            }
        },
        "hetero_feature_selection_0": {
            "module": "HeteroFeatureSelection",
            "input": {
                "data": {"data": [
                        "hetero_feature_binning_0.train"
                    ]
                },
                "isometric_model": [
                    "hetero_feature_binning_0.hetero_feature_binning"
                ]
            },
            "output": {
                "data": ["eval"],
                "model": ["selected"]
            }
        },
        "evaluation_0": {
            "module": "Evaluation",
            "input": {
                "data": {
                    "data": ["hetero_feature_selection_0.eval"]
                }
            },
            "output": {
                "data": ["evaluate"]
            }
        }
    }
}
'''

config = '''
{
    "initiator": {
        "role": "guest",
        "party_id": 10000
    },
    "job_parameters": {
        "work_mode": 1,
        "job_type": "predict",
        "model_id": "arbiter-9999#guest-10000#host-9999#model",
        "model_version": "2020060802475836992436"
    },
    "role": {
        "guest": [10000],
        "host": [9999],
        "arbiter": [9999]
    },
    "role_parameters": {
        "guest": {
            "args": {
                "data": {
                    "eval_data": [{"name": "breast_b", "namespace": "fate_flow_test_breast"}]
                }
            }
        },
        "host": {
            "args": {
                "data": {
                    "eval_data": [{"name": "breast_a", "namespace": "fate_flow_test_breast"}]
                }
            }
        }
    }
}
'''

In [22]:
response = manager.submit_job(json.loads(pipline), json.loads(config))
manager.prettify(response, True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    "job_id": jobId
}

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020063006573868964950&role=guest&party_id=9999",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020063006573868964950/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020063006573868964950/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020063006573868964950",
        "model_info": {
            "model_id": "arbiter-9999#guest-9999#host-9999#model",
            "model_version": "2020063006531172788744"
        }
    },
    "jobId": "2020063006573868964950",
    "retcode": 0,
    "retmsg": "success"
}


In [23]:
print(query_condition)

{'job_id': '2020063006573868964950'}


In [24]:
model_id, model_version = "", ""
final_status = manager.query_job_status(query_condition)

if final_status == "failed":
    print("Failed")
    manager.prettify(job_detail, True)
    response = manager.fetch_job_log(jobId)
    raise Exception("Failed to run the job")
if final_status == "success":
    print("Success")
    manager.prettify(response, True)
    output = json.loads(response.content)
    model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]

Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: 

In [25]:
manager.track_component_output_data("2020060806580950942753", "guest", "10000", "hetero_lr_0")

Success!
{
    "data": [
        [
            "8",
            0,
            1,
            0.7046115406302281,
            {
                "0": 0.29538845936977187,
                "1": 0.7046115406302281
            },
            "predict"
        ],
        [
            "17",
            0,
            1,
            0.9564225652538682,
            {
                "0": 0.04357743474613185,
                "1": 0.9564225652538682
            },
            "predict"
        ],
        [
            "26",
            0,
            1,
            0.9661537574810894,
            {
                "0": 0.03384624251891055,
                "1": 0.9661537574810894
            },
            "predict"
        ],
        [
            "35",
            0,
            0,
            0.19285719929095904,
            {
                "0": 0.807142800709041,
                "1": 0.19285719929095904
            },
            "predict"
        ],
        [
            "44",
            

<Response [200]>